# Problem set 2 -  Hoarding and Structural Breaks

### Author: David Henning

### Part I: Hoarding

##### 1) Compute: if the desired number of children is 5, what is the number of births when Infant Mortality Rate (IMR) is 0.5? 0.3? 0.1? 0.05? 0.01? 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.api import OLS, tsa

In [ ]:
#Define the parameters
N = 5
l = np.array([0.5, 0.3, 0.1, 0.05, 0.01])
Birth = np.empty_like(l)

#Loop to calculate the number of births given target number of kids and mortality rate
for i in range(len(l)):
    Birth[i] = N / (1-l[i])
    
Birth

##### 2) The total fertility rate of the world went from around 6-7 to around 2. Given your computations what can we say about hoarding as a theory to explain the fertility transition? 

I don't really understand this question...

### Part II: Structural Breaks

In [ ]:
np.random.seed(1)
T = 100
y = np.zeros(T)                        # To store values

for t in range(0, T):
    y[t] = np.random.uniform(1, 10)

In [ ]:
plt.plot(y)
plt.title('Random data points $y$')
plt.xlabel('t')
plt.show()

In [ ]:
#Generate a dataframe to work with from now on
x = list(range(100))
df = pd.DataFrame(x,columns=['x'])
df['y'] = pd.DataFrame(y, columns=['y'])
df['constant'] = 1

##### 3) Suppose $y$ increases linearly with time $t$ , with some error, and has no break in trend. What is the estimated break? 

In [ ]:
# Set parameters
J = df.shape[1] - 1
k = df.shape[1] - 1
chow = np.zeros(90)

# Run pooled OLS model
model = OLS(df.y, df[['x', 'constant']]).fit()
RSS = model.ssr

In [ ]:
plt.plot(model.fittedvalues)
plt.plot(df.y)
plt.show()

In [ ]:
for i in range(0,90):
    t = i + 5
    
    df1 = df[:t]
    df2 = df[t:]
    
    N1 = df1.shape[0]
    N2 = df2.shape[0]

    model1 = OLS(df1.y, df1[['x', 'constant']]).fit()
    RSS1 = model1.ssr

    model2 = OLS(df2.y, df2[['x', 'constant']]).fit()
    RSS2 = model2.ssr

    chow[i] = ((RSS-(RSS1+RSS2))/J)/((RSS1+RSS2)/(N1+N2-2*k))


In [ ]:
df3 = df
df3['chow'] = pd.DataFrame(chow, columns=['chow'])
df3['chow'] = df.chow.shift(5)

In [ ]:
df3.plot(x='x', y='chow')

In [ ]:
df3.idxmax(axis=0)['chow'] #display the time at which chow is maximized

##### 4) Supppose  $y$ is stationary and has a break at $t=30$ . What is the estimated break? What happens if the variance increases over time? 

In [ ]:
#Define dataframe for AR(1) process
df4 = df[['y', 'x', 'constant']]
ε = np.random.normal(scale=1, size=T)  # Draw random error terms

df4['y_lag'] = df4.y.shift(1)          # Create lagged version of dependent variable
df4['break'] = (df.x >= 30).astype(int)   # Create dummy for break
df4['y_lag_break'] = df4['y_lag'] * df4['break']  #multiply dummy with lagged to allow for differential slopes
df4['error'] = ε                       # Add error term
df5 = df4[['x','y', 'break', 'constant']]
df4 = df4[1:]                          # Drop first observation

In [ ]:
# Set parameters
J = df4.shape[1] - 1
k = df4.shape[1] - 1
chow = np.zeros(90)

# Run pooled OLS model
model = OLS(df4.y, df4[['constant', 'y_lag', 'break', 'y_lag_break', 'error']]).fit()
RSS = model.ssr

In [ ]:
plt.plot(model.fittedvalues)
plt.plot(df4.y)
plt.show()

In [ ]:
for i in range(0,90):
    t = i + 5
    
    df1 = df4[:t]
    df2 = df4[t:]
    
    N1 = df1.shape[0]
    N2 = df2.shape[0]

    model1 = OLS(df1.y, df1[['constant', 'y_lag', 'break', 'y_lag_break', 'error']]).fit()
    RSS1 = model1.ssr

    model2 = OLS(df2.y, df2[['constant', 'y_lag', 'break', 'y_lag_break', 'error']]).fit()
    RSS2 = model2.ssr

    chow[i] = ((RSS-(RSS1+RSS2))/J)/((RSS1+RSS2)/(N1+N2-2*k))

In [ ]:
df4 = df
df4['chow'] = pd.DataFrame(chow, columns=['chow'])
df4['chow'] = df4.chow.shift(5)
df4.plot(x='x', y='chow')

In [ ]:
df4.idxmax(axis=0)['chow'] #display the time at which chow is maximized

##### 5) Supppose  $y$ increases exponentially with time $t$, and has a break at $t=30$ . What is the estimated break? 

In [ ]:
#Define dataframe for AR(1) process
df5['y'] = np.log(df5['y'])
df5['x_break'] = df5['x'] * df5['break']  #multiply dummy with lagged to allow for differential slopes
df5 = df5[['y', 'x', 'break', 'x_break', 'constant']]

In [ ]:
# Set parameters
J = df5.shape[1] - 1
k = df5.shape[1] - 1
chow = np.zeros(90)

# Run pooled OLS model
model = OLS(df5.y, df5[['break', 'x', 'x_break', 'constant']]).fit()
RSS = model.ssr

In [ ]:
plt.plot(model.fittedvalues)
plt.plot(df5.y)
plt.show()

In [ ]:
for i in range(0,90):
    t = i + 5
    
    df1 = df5[:t]
    df2 = df5[t:]
    
    N1 = df1.shape[0]
    N2 = df2.shape[0]

    model1 = OLS(df1.y, df1[['break', 'x', 'x_break', 'constant']]).fit()
    RSS1 = model1.ssr

    model2 = OLS(df2.y, df2[['break', 'x', 'x_break', 'constant']]).fit()
    RSS2 = model2.ssr

    chow[i] = ((RSS-(RSS1+RSS2))/J)/((RSS1+RSS2)/(N1+N2-2*k))

In [ ]:
df5['chow'] = pd.DataFrame(chow, columns=['chow'])
df5['chow'] = df5.chow.shift(4)
df5.plot(x='x', y='chow')

In [ ]:
df5.idxmax(axis=0)['chow'] #display the time at which chow is maximized

##### 6) Supppose  $y$ stationary and has breaks at $t=30$ and $t=60$ . What is the estimated break? 

In [ ]:
#Define dataframe for AR(1) process
df6 = df[['y', 'x', 'constant']]
ε = np.random.normal(scale=1, size=T)  # Draw random error terms

df6['y_lag'] = df6.y.shift(1)          # Create lagged version of dependent variable
df6['break1'] = (df6.x >= 30).astype(int)   # Create dummy for break
df6['break2'] = (df6.x >= 60).astype(int)   # Create dummy for break 2
df6['y_lag_break1'] = df6['y_lag'] * df6['break1']  #multiply dummy with lagged to allow for differential slopes
df6['y_lag_break2'] = df6['y_lag'] * df6['break2']  #multiply dummy with lagged to allow for differential slopes
df6['error'] = ε                       # Add error term
df6 = df6[1:]                          # Drop first observation

In [ ]:
# Set parameters
J = df6.shape[1] - 1
k = df6.shape[1] - 1
chow = np.zeros(90)

# Run pooled OLS model
model = OLS(df6.y, df6[['constant', 'y_lag', 'break1', 'break2', 'y_lag_break1', 'y_lag_break2', 'error']]).fit()
RSS = model.ssr

In [ ]:
plt.plot(model.fittedvalues)
plt.plot(df6.y)
plt.show()

In [ ]:
for i in range(0,90):
    t = i + 5
    
    df1 = df6[:t]
    df2 = df6[t:]
    
    N1 = df1.shape[0]
    N2 = df2.shape[0]

    model1 = OLS(df1.y, df1[['constant', 'y_lag', 'break1', 'break2', 'y_lag_break1', 'y_lag_break2', 'error']]).fit()
    RSS1 = model1.ssr

    model2 = OLS(df2.y, df2[['constant', 'y_lag', 'break1', 'break2', 'y_lag_break1', 'y_lag_break2', 'error']]).fit()
    RSS2 = model2.ssr

    chow[i] = ((RSS-(RSS1+RSS2))/J)/((RSS1+RSS2)/(N1+N2-2*k))

In [ ]:
df6['chow'] = pd.DataFrame(chow, columns=['chow'])
df6['chow'] = df6.chow.shift(5)
df6.plot(x='x', y='chow')

In [ ]:
df6.idxmax(axis=0)['chow'] #display the time at which chow is maximized